In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
def root_mse(x, y):
    if len(x) != len(y):
        return "Error: The two arguments must have the same length"
    n = len(x)
    sq_err = [(x[i] - y[i]) ** 2 for i in range(n)]
    mse = sum(sq_err) / n
    # mse = np.square(np.subtract(x, y)).mean()
    return np.sqrt(mse)


def multi_rmse(observed, predicted):
    # Ensure the lists are of the same dimensions
    if len(observed) != len(predicted):
        raise ValueError("The number of rows in observed and predicted arrays must be the same")
    
    num_rows = len(observed)
    if num_rows == 0:
        raise ValueError("The input arrays must not be empty")
    
    num_columns = len(observed[0])
    for row in observed:
        if len(row) != num_columns:
            raise ValueError("All rows in the observed array must have the same number of columns")
    for row in predicted:
        if len(row) != num_columns:
            raise ValueError("All rows in the predicted array must have the same number of columns")
    
    # Calculate RMSE for each column
    columnwise_rmse = []
    for col in range(num_columns):
        squared_errors = [(observed[row][col] - predicted[row][col]) ** 2 for row in range(num_rows)]
        mean_squared_error = sum(squared_errors) / num_rows
        rmse = math.sqrt(mean_squared_error)
        columnwise_rmse.append(rmse)
    
    return columnwise_rmse


In [11]:
# Generating a sample multivariate time series dataset
def generate_data(num_samples=1000, num_features=3, timesteps=10):
    np.random.seed(42)
    data = np.random.rand(num_samples, num_features)
    return data

# Preprocessing the data
def preprocess_data(data, timesteps=10):
    X, y = [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:i+timesteps])
        y.append(data[i+timesteps])
    X, y = np.array(X), np.array(y)
    
    # Normalizing the data
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
    y = scaler.transform(y)
    
    return X, y, scaler

# Build the CNN model
# def build_model(timesteps, num_features):
#     model = Sequential()
#     model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(timesteps, num_features)))
#     model.add(Flatten())  
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(num_features))
#     model.compile(optimizer='adam', loss='mse')
#     return model

def build_model(timesteps, num_features):
    model = Sequential()
    
    # First Conv1D layer
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(timesteps, num_features)))
    model.add(Dropout(0.2))  # Optional: Add dropout to prevent overfitting
    
    # Second Conv1D layer
    model.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
    model.add(Dropout(0.2))  # Optional: Add dropout to prevent overfitting
    
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_features))
    
    model.compile(optimizer='adam', loss='mse')
    return model

def plot_series(series, attr):
    for i in range(len(attr)):
        title = "Plot of "+str(attr[i])
        actual = series.iloc[:,i]
        plt.title(title)
        plt.xlabel("Timestep")
        plt.ylabel(attr[i])
        plt.plot(actual)
        plt.show()

# Main function
if __name__ == "__main__":
    # Parameters
    num_samples = 1000
    # num_features = 3
    timesteps = 5
    mse_list = []
    rmse_list = []
    mae_list = []
    rsqr_list = []

    # Generate and preprocess data
    # data = generate_data(num_samples, num_features)
    # data = pd.read_csv("waterTank_Golden_reduced.csv", index_col=0, header=0)
    # data.index.freq= 'ms'
    file = "phy_cps.csv"
    df = pd.read_csv(file, index_col=0)
    data = df.values
    features = df.columns
    num_features = len(features)
    X, y, scaler = preprocess_data(data, timesteps)
    
    # Split the data into training and testing sets
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    # Build and train the model
    model = build_model(timesteps, num_features)
    model.summary()
    # model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)
    model.fit(X_train, y_train, epochs=200, batch_size=16, validation_split=0.2)
    
    # Evaluate the model
    loss = model.evaluate(X_test, y_test)
    print(f"Test Loss: {loss}")
    
    # Make predictions
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    real_y_test = scaler.inverse_transform(y_test)
    # # Example: Print the first 5 predictions
    # print("First 5 Predictions:")
    # print(predictions[:5])

    for i in range(1, num_features+1):
        mse = mean_squared_error(predictions[i], real_y_test[i])
        mse_list.append(mse)
        print(f'Mean Squared Error: {mse}')

        rmse = np.sqrt(mse)
        rmse_list.append(rmse)
        print(f'Root Mean Squared Error: {rmse}')

        mae = mean_absolute_error(predictions[i], real_y_test[i])
        mae_list.append(mae)
        print(f'Mean Absolute Error: {mae}')

        r_sqr = r2_score(predictions[i], real_y_test[i])
        rsqr_list.append(r_sqr)
        print(f'R2 Score: {r_sqr}')
    
# for i in range(len(data[0])):
#     title = "Plot of "
#     actual = data[:,i]
#     plt.title(title)
#     plt.xlabel("Timestep")
#     # plt.ylabel(attr[i])
#     plt.plot(actual)
#     plt.show()
    # print("The mean squared errors are as follows:")
    # print(mean_squared_error(predictions, y_test))
    
    # print("The root mean squared errors are as follows:")
    # print(root_mse(predictions, y_test))
    
    # print("The mean absolute errors are as follows:")
    # print(mean_absolute_error(predictions, y_test))
    
    # print("The R2-score values are as follows:")
    # print(r2_score(predictions, y_test))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_16 (Conv1D)          (None, 4, 64)             832       
                                                                 
 dropout_16 (Dropout)        (None, 4, 64)             0         
                                                                 
 conv1d_17 (Conv1D)          (None, 3, 32)             4128      
                                                                 
 dropout_17 (Dropout)        (None, 3, 32)             0         
                                                                 
 flatten_8 (Flatten)         (None, 96)                0         
                                                                 
 dense_16 (Dense)            (None, 50)                4850      
                                                                 
 dense_17 (Dense)            (None, 6)                

In [ ]:
print(mean_squared_error(predictions[1], y_test[1]))
print(multi_rmse(predictions, y_test))

In [ ]:
print(mse_list)
print(rmse_list)
print(mae_list)
print(rsqr_list)

In [ ]:
def ListShape(list):
    rows = len(list)
    # if True:
    if isinstance(list[0], int):
        return "Error: object is a scalar/number"
    cols = len(list[0]) if rows > 0 else 0
    return rows, cols

print(ListShape(predictions))

In [ ]:
# print(multi_rmse(y_test, predictions))
# print(multi_rmse(predictions, y_test))
# [0.32743663 0.35012764 0.32282214]

In [ ]:
data = pd.read_csv(file, index_col=0)
num_features = len(data.columns)
num_features

In [ ]:
data = pd.read_csv(file, index_col=0)
num_features = len(data.columns)
data.columns

In [ ]:
# Plot the predictions
n = ListShape(predictions)[1]
for i in range(n):
    plt.figure(figsize=(10, 6))
    plt.plot(predictions[:, i], label=f'predictions - Feature {i+1}')
    plt.plot(real_y_test[:, i], label=f'y_test - Feature {i+1}')
    plt.title(f'Plot of {data.columns[i]}')
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

In [ ]:
#plot the entire dataset
for i in range(len(features)):
    title = "Prediction of {}".format(features[i])
    plt.title(title)
    plt.xlabel("Timestep")
    plt.ylabel("Values")
    plt.plot(real_y_test[:,i], label="actual")
    # plt.plot(predictions[:,i], label="Forecast")
    plt.legend()
    plt.show()

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten
import time

# Load data from CSV file
def load_data(csv_file):
    data = pd.read_csv(csv_file, index_col=0)
    return data

# Prepare data for the model
def prepare_data(data, n_timesteps):
    X, y = [], []
    for i in range(len(data) - n_timesteps):
        X.append(data.iloc[i:i+n_timesteps].values)
        y.append(data.iloc[i+n_timesteps].values)
    X = np.array(X)
    y = np.array(y)
    return X, y

# Parameters
csv_file = 'phy_cps.csv'  # Replace with the path to your CSV file
n_timesteps = 10  # Number of timesteps to use for prediction

# Load and prepare data
data = load_data(csv_file)
n_features = data.shape[1]
X, y = prepare_data(data, n_timesteps)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = X_train.reshape(-1, n_features)
X_test = X_test.reshape(-1, n_features)

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

X_train = X_train.reshape(-1, n_timesteps, n_features)
X_test = X_test.reshape(-1, n_timesteps, n_features)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# Build the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_features))

model.compile(optimizer='adam', loss='mse')

# Train the model and measure the time
start_time = time.time()
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)
end_time = time.time()
training_time = end_time - start_time

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and actual values
y_pred = scaler_y.inverse_transform(y_pred)
y_test = scaler_y.inverse_transform(y_test)

# Evaluate the model for each feature
results = {}
for i in range(n_features):
    mae = mean_absolute_error(y_test[:, i], y_pred[:, i])
    mse = mean_squared_error(y_test[:, i], y_pred[:, i])
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test[:, i], y_pred[:, i])
    results[f'Feature {i+1}'] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2 Score': r2
    }

# Print the results
print(f"Training Time: {training_time:.2f} seconds")
for feature, metrics in results.items():
    print(f"\n{feature}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")



Epoch 1/100
49/49 [==============================] - 1s 5ms/step - loss: 0.0441 - val_loss: 0.0092
Epoch 2/100
49/49 [==============================] - 0s 3ms/step - loss: 0.0089 - val_loss: 0.0055
Epoch 3/100
49/49 [==============================] - 0s 3ms/step - loss: 0.0068 - val_loss: 0.0047
Epoch 4/100
49/49 [==============================] - 0s 3ms/step - loss: 0.0058 - val_loss: 0.0042
Epoch 5/100
49/49 [==============================] - 0s 3ms/step - loss: 0.0053 - val_loss: 0.0043
Epoch 6/100
49/49 [==============================] - 0s 3ms/step - loss: 0.0046 - val_loss: 0.0037
Epoch 7/100
49/49 [==============================] - 0s 3ms/step - loss: 0.0047 - val_loss: 0.0037
Epoch 8/100
49/49 [==============================] - 0s 3ms/step - loss: 0.0039 - val_loss: 0.0044
Epoch 9/100
49/49 [==============================] - 0s 5ms/step - loss: 0.0041 - val_loss: 0.0030
Epoch 10/100
49/49 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0029
Epoch 11/